In [ ]:
# See: https://github.com/maladeep/Name-Matching-In-Python/blob/master/Surprisingly%20Effective%20Way%20To%20Name%20Matching%20In%20Python.ipynb

In [27]:
import pandas as pd 
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import numpy as np
from dynaconf import LazySettings
from dynaconf.utils.boxing import DynaBox
from scipy.sparse import csr_matrix
from typing import List
import sparse_dot_topn.sparse_dot_topn as ct  #Cosine Similarity
import time
#pd.set_option('display.max_colwidth', -1)

In [2]:
config_file = "/home/tiziano/workspaces/fantasAi_football/config/conf.yaml"
config_mode = 'default'

In [3]:
def data_ingestion_basics(params: DynaBox, table: str, duplicate_key_err:str='raise') -> pd.DataFrame:
    """Performs some basic harmonization steps on a input dataframe.

    Note that after the basic harmonization the columns will have the names
    stated in settings.COLS, and not those appearing in the input data.

    Args:
        table (pd.DataFrame): id of the table to lead, it must appear in
            config file
        duplicate_key_err ('raise' or 'drop'): how to handle duplicated key
        errors. If 'raise' an error will be return in case of duplicated keys,
        if 'drop' all duplicated rows will be dropped from data.

    Raises:
        ValueError: If table registry dataframe keys are missing
        KeyError: If table registry dataframe keys are duplicated

    Returns:
        pd.DataFrame: dataframe in input, harmonized
    """
    # Load parameters
    t_par = params[table]
    t_cols_dict = t_par["COLS"].to_dict()
    keys: List[str] = [t_par["COLS"][c] for c in t_par.KEY]
    cols: List[str] = list(t_cols_dict.values())
    dtype_map = {
        c_name: params["FEATURES"]["DTYPES"][c]
        for c, c_name in t_cols_dict.items()
    }
    name_conversion = {
        c_from: params["FEATURES"][c] for c, c_from in t_cols_dict.items()
    }
    
    path = os.path.join(
        params["PATHS"]["ROOT_FOLDER"], 
        params["PATHS"]["INPUT"]["FOLDER"],
         params["PATHS"]["INPUT"][table]
    )
    data: pd.DataFrame = pd.read_csv(path)

    # Keep only selected columns
    data = data[cols]

    # Ugly but functional call to ensure correct type conversion
    data = data.convert_dtypes().astype(dtype_map, errors='ignore').convert_dtypes()  # type: ignore

    # Ensure that the product registry dataframe keys are valid
    if not data[keys].notna().all(axis=1).all():
        raise ValueError(f"{table} dataframe keys are missing")
    if not data.value_counts(keys).eq(1).all():
        if duplicate_key_err == 'raise':
            raise KeyError(f"{table} keys are duplicated")
        else:
            print(f"WARN: dupliated keys will be removed from {table}")
            data = data.drop_duplicates(subset=keys, keep=False)

    # Sort product product registry by DIVISION,PRODUCT and reset index
    data = data.sort_values(keys).reset_index(drop=True)

    data = data.rename(columns=name_conversion)

    if "FILTER" in params:
        if table in params["FILTER"]:
            for column, values in params["FILTER"].get(table).to_dict().items():
                column_name = params["FEATURES"].get(column)
                data = data.loc[data[column_name].isin(values)]

    return data

In [4]:
params = LazySettings(settings_files=[config_file])
params = params[config_mode]

In [5]:
players = data_ingestion_basics(params, "PLAYERS")
votes_ita = data_ingestion_basics(params, "VOTES_ITA", duplicate_key_err='drop')

players['date_of_birth'] = pd.to_datetime(players['date_of_birth'], errors='coerce')



WARN: dupliated keys will be removed from VOTES_ITA


In [6]:
def ngrams(string, n=3):
    string = re.sub(r'[,-./]|\sBD',r'', string)
    ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in ngrams]

In [7]:
names = pd.concat(
    [players[params["FEATURES"]["NAME"]], 
    votes_ita[params["FEATURES"]["PIANETAFANTA_NAME"]]],
    ignore_index=True
    )
names = names.str.lower()

In [129]:

# After having each words split (token or  lemmas (n-gram generated items) ) into a vector and
# Scikit-learn’s  Tfidfvectorizer aim to do the same thing, which is to convert a collection of raw documents to a matrix of TF-IDF features. 
# Generate the matrix of TF-IDF (Term Frequency-Inverse Document frequency)values for each
#vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams)
vectorizer = TfidfVectorizer(min_df=1, analyzer='word')
vectorizer = vectorizer.fit(names)

tm_name_club = players[params["FEATURES"]["PRETTY_NAME"]] + " " + players[params["FEATURES"]["CLUB"]] non va bene id devi mettere nome
tm_name_club = tm_name_club.str.lower()
pf_names_club = votes_ita[params["FEATURES"]["PIANETAFANTA_NAME"]] + " " + votes_ita[params["FEATURES"]["PIANETAFANTA_TEAM"]]
pf_names_club = pf_names_club.str.lower()
names = pd.concat(
    [tm_name_club, pf_names_club],
    ignore_index=True
    )
    
tm_names = vectorizer.transform(tm_name_club) 
test_names = vectorizer.transform(["ronaldo", "mancini"]) 
pf_names = vectorizer.transform(pf_names_club)

In [145]:
tm_name_club

0               miroslav klose 398
1                      xandao 2700
2                  wes morgan 1003
3              thomas azevedo 2727
4        cristiano figueiredo 1085
                   ...            
22616              marc bartra 150
22617        simone benedetti 1390
22618              tano bonnin 331
22619              gijs jasper 202
22620           marko scepovic 126
Length: 22621, dtype: string

In [135]:
votes_ita.iloc[0]

season                        2
pt_name            ABBATE Matte
pt_team                piacenza
pt_role                       D
played_matches                6
started_matches               6
quotation                   3.2
vote_gazzetta              5.83
vote_corriere               0.0
vote_tuttosport             0.0
vote_average               5.83
goals                         0
yellow_cards                  0
red_cards                     0
assists                       0
Name: 0, dtype: object

In [136]:
cosine_similarities = linear_kernel(pf_names, tm_names)

In [137]:
cosine_similarities.shape

(10650, 22621)

In [138]:
# Find the top 5 related documents
related_docs_indices = cosine_similarities.argsort()[1,-5:-1]

In [141]:
related_docs_indices.shape

(4,)

In [144]:
players.iloc[related_docs_indices[:]]['pretty_name']

15819    Milan Mitrovic
4467     Milan Skriniar
13129      Milan Badelj
20163       Guido Milan
Name: pretty_name, dtype: string